In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mlp import MLP, Layer, InputLayer, OutputLayer, MultinoulliML, Adam

In [ ]:
import os

class Iterator:
    def __iter__(self):
        return self

class chain(Iterator):
    def __init__(self, *its):
        self.its = iter([iter(it) for it in its])
        self.it = None

    def __next__(self):
        if self.it is None:
            self.it = next(self.its)
            return self.__next__()
        else:
            try:
                return next(self.it)
            except StopIteration:
                self.it = None
                return self.__next__()
            
class chunk(Iterator):
    def __init__(self, it, n):
        self.it = it
        self.n = n

    def __next__(self):
        if self.n > 0:
            self.n -= 1
            return next(self.it)
        else: raise StopIteration
        
class chunked_it(Iterator):
    def __init__(self, it, chunksize):
        self.it = it
        self.chunksize = chunksize
        self.chunk = None

    def __next__(self):
        if self.chunk is None:
            _next = next(self.it)
            self.chunk = chunk(chain([_next], self.it), self.chunksize)
            return self.chunk
        else:
            list(self.chunk)
            self.chunk = None
            return self.__next__()
        
def file_it(path, skiplines=0):
    with open(path) as f:
        for _ in range(skiplines):
            f.readline()
        yield from f

def file_it_inf(path, skiplines=0):
    with open(path) as f:
        while 1:
            for _ in range(skiplines):
                f.readline()
            yield from f
            f.seek(0, os.SEEK_SET)

def gen_batch(c):
    arr = np.loadtxt(c, dtype=int, delimiter=',')
    return arr[:, 1:]/255, arr[:, 0]

In [ ]:
it = map(gen_batch, chunked_it(file_it("../data/train.csv", skiplines=1), chunksize=10))
valset = next(it)

In [ ]:
network = MLP(
    [
        InputLayer(28*28),
        Layer(16),
        Layer(16),
        OutputLayer(10)
    ],
    model=MultinoulliML(),
    optimizer=Adam(eps=0.5)
)

network.init_training(batchsize=10, validation_set=valset)

In [ ]:
for batch in it:
    network.train_minibatch(batch)

In [ ]:
it = map(
        lambda line: np.fromstring(line, sep=",", dtype=np.uint8),
        file_it("../data/test.csv", skiplines=1),
)


In [ ]:
data = next(it)
n = network.feedforward(data/255)
print(n)
plt.imshow(data.reshape((28,28)), cmap="gray", vmin = 0, vmax = 255)

In [ ]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

In [ ]:
x = np.eye(10,10)
window = sliding_window_view(x, (2,2))

In [ ]:
window.shape

In [ ]:
np.array(1.)*2